##### 보유종목의 현재가 갱신하기

In [94]:
import pandas   as pd
df = pd.read_excel("관심종목.xlsx",sheet_name="Sheet2")
df = df.iloc[9:,]   # 테이블의 sheet 상의 위치
df.columns = ["종목명","종목코드","잔고","매입금액","평단가","현재가","평가금액","손익","손익율"]
df = df.reset_index(drop=True)
df.head(20)

,종목명,종목코드,잔고,매입금액,평단가,현재가,평가금액,손익,손익율
0,삼성전자,005930,15,1047000,69800,68800,NaN,NaN,NaN
1,와이지-원,019210,142,986900,6950,5810,NaN,NaN,NaN
2,황금에스티,032560,116,994120,8570,8330,NaN,NaN,NaN
3,한국항공우주,047810,18,984600,54700,48850,NaN,NaN,NaN
4,인탑스,049070,29,990350,34150,29900,NaN,NaN,NaN
5,한미글로벌,053690,32,974400,30450,25600,NaN,NaN,NaN
6,어보브반도체,102120,91,990080,10880,9770,NaN,NaN,NaN
7,TIGER 미국S&P500레버리지(합성),225040,309,10055555,32542,29685,NaN,NaN,NaN
8,KODEX 미국나스닥100레버리지(합성),409820,210,1925500,9169,7305,NaN,NaN,NaN
9,KODEX 레버리지,122630,273,5147415,18855,17075,NaN,NaN,NaN


In [95]:
tickers = df["종목코드"].to_list()
tickers

['005930',
 '019210',
 '032560',
 '047810',
 '049070',
 '053690',
 '102120',
 '225040',
 '409820',
 '122630',
 '225040',
 '005930']

In [96]:
from  pykrx import stock

sdate = '20230804'   # 기준일자 세팅
tickers_set = set(tickers)
data = dict()
for ticker in tickers_set :
    try :
        if  ticker in ['225040','409820','122630'] :
            temp_df = stock.get_etf_ohlcv_by_date(sdate,sdate, ticker)  # ETF
        else :
            temp_df = stock.get_market_ohlcv(sdate,sdate,ticker)
    except ValueError :
        print(f'{ticker} ValueError')
        continue
    else :
        data[ticker] = temp_df["종가"][0]

data

{'005930': 68500,
 '409820': 7370,
 '049070': 30700,
 '053690': 26450,
 '122630': 17110,
 '102120': 9700,
 '032560': 8180,
 '225040': 29755,
 '019210': 5920,
 '047810': 49250}

In [97]:
from openpyxl import load_workbook

wb = load_workbook("관심종목.xlsx")
ws = wb["Sheet2"]

for row in ws['A11:I22'] :   # 데이터가 위치하는 sheet 상의 위치
    print(data[row[1].value])
    row[5].value = data[row[1].value]

wb.save("관심종목.xlsx")    

68500
5920
8180
49250
30700
26450
9700
29755
7370
17110
29755
68500


##### 이하 잡동사니...

In [83]:
df_merged = df.merge(df_current,on="종목코드", how="left")
df_merged = df_merged.rename(columns={"현재가_y":"현재가"})
df_merged = df_merged.drop("현재가_x", axis=1)
df_merged

,종목명,종목코드,잔고,매입금액,평단가,평가금액,손익,손익율,현재가
0,삼성전자,005930,15,1047000,69800,1047000,0,0,68800
1,와이지-원,019210,142,986900,6950,816500,-170400,-0.172662,5810
2,황금에스티,032560,116,994120,8570,954680,-39440,-0.039673,8330
3,한국항공우주,047810,18,984600,54700,883800,-100800,-0.102377,48850
4,인탑스,049070,29,990350,34150,885950,-104400,-0.105417,29900
5,한미글로벌,053690,32,974400,30450,804800,-169600,-0.174056,25600
6,어보브반도체,102120,91,990080,10880,878150,-111930,-0.113051,9770
7,TIGER 미국S&P500레버리지(합성),225040,309,10055555,32542,9206655,-848823,-0.084413,29685
8,KODEX 미국나스닥100레버리지(합성),409820,210,1925500,9169,1542450,-383040,-0.19893,7305
9,KODEX 레버리지,122630,273,5147415,18855,4673760,-473655,-0.092018,17075


In [38]:
tickers = [row[1].value for row in ws['A11:I22'] ]
print(tickers)


['005930', '019210', '032560', '047810', '049070', '053690', '102120', '225040', '409820', '122630', '225040', '005930']


In [21]:
from  pykrx import stock
import pandas   as pd

In [32]:
df = stock.get_market_ohlcv('20230803','20230803', "005930")
df

,시가,고가,저가,종가,거래량,거래대금,등락률
날짜,,,,,,,
2023-08-03,69900,70200,68500,68800,20087090,1393671498850,-1.57


In [53]:
for ticker in tickers :
    try :
        if  ticker in ['225040','409820','122630'] :
            df = stock.get_etf_ohlcv_by_date('20230803','20230803', ticker)  # ETF
        else :
            df = stock.get_market_ohlcv('20230803','20230803',ticker)
    except ValueError :
        print(f'{ticker} ValueError')
        continue
    else :
        print(ticker,df["종가"][0])
    # row[5].value = df['종가']

005930 68800
019210 5810
032560 8330
047810 48850
049070 29900
053690 25600
102120 9770
225040 29685
409820 7305
122630 17075
225040 29685
005930 68800


In [51]:

ticker_list = stock.get_market_ticker_list(market='KOSPI')
ticker_list.index('225040')

ValueError: '225040' is not in list